In [1]:
#Required Libraries...

from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import requests
import pandas as pd

In [2]:
#...GLOBAL VARIABLES...

#Columns Definition for DataFrame...
column_names = ['N°', 'Nombre', 'Imagen', 'Tipo', 'Fuerza', 'Coste de oro', 'Raza', 'Frecuencia', 'Edición', 'Habilidad', 'Código', 'Arte', 'Texto épico']

# Decks List "Primera Era"...
decksFirstEra = {'El_Reto', 'Mundo_G%C3%B3tico', 'La_Ira_del_Nahual', 'Ragnarok' , 'La_Cofrad%C3%ADa', 'Esp%C3%ADritu_de_Drag%C3%B3n'}

# Filename to export file...
fileName = ''

In [3]:
#Initial Pages for Scrapping...
root = 'https://myl.fandom.com'
deckList = f'{root}/es/wiki/Categor%C3%ADa:Lista_de_Cartas'

#Opening URL that contain List of Decks...
page_html = requests.get(deckList)
page_soup = soup(page_html.text, "lxml")

In [4]:
#Scrapping to get URLs for each Deck...
box = page_soup.find('div' , class_='category-page__members')
deckList = box.findAll('a', class_= 'category-page__member-link', href=True)

In [5]:
#Getting URL for each Deck in the List to access the card list per deck...
deckLinks = []
for link in deckList:
        deckLinks.append(link['href'])

In [6]:
#Filtering URLs to get only "Primera Era Decks"...
decksLinkFE = []
for link in deckLinks:
    if any(i in link for i in decksFirstEra):
        decksLinkFE.append(link)

In [22]:
#Opening URL of specific Deck:
#-------------------------------------
# 0 El Reto
# 1 El Reto Extension
# 2 Espíritu del Dragón
# 3 Cofradía
# 4: La ira del nahual
# 5: Mundo Gótico
# 6: Mundo Gótico Extension
# 7: Ragnarok
#--------------------------------------

cardList = f'{root}{decksLinkFE[7]}'

#FileName after exporting
fileName = cardList.replace('https://myl.fandom.com/es/wiki/', '')

#Opening URL of specific Deck
page_html = requests.get(cardList)
page_soup = soup(page_html.text, "lxml")
page_table = page_soup.find_all('table')[1] 

cardLinks= []

#Scrapping to get card URLs...

for row in page_table.find_all('tr'):
    cardRow = []
    for td in row.find_all('td'):
        td_check = td.find('a')
        if td_check is not None:
            link = td.a['href']
            cardRow.append(link)
        else:
            not_link = ''.join(td.stripped_strings)
            if not_link == '':
                 not_link = None
            cardRow.append(not_link)
    cardLinks.append(cardRow)


cardLinkList = []

for i in range(len(cardLinks)-1):
    cardLinkList.append(cardLinks[i+1][1])
    
#print(cardLinkList)


In [19]:
# Iteration to get all the cards information in the specific deck...

cardDetail =[]

title = ''
cardImage = ''
cardDescription = []
cardHability = ''
cardCode = ''
cardArtist = ''
cardHistory = ''

for i in range(len(cardLinkList)):
    #Opening URL of specific Card
    cardList = f'{root}{cardLinkList[i]}'
    #print(cardList)
    page_html = requests.get(cardList)
    page_soup = soup(page_html.text, "lxml")
    card_tables = page_soup.findAll('table', class_='table_black')

# Primera Tabla [0]... table_header contiene nombre de Carta 
# table_image contiene imagen
# table_description contiene: Tipo, Coste, Edición, Frecuencia, Fuerza, Raza
# card_with_hability: To identify if card has hability

    table_header = card_tables[0].find('tr', class_='table_head_black')
    table_image = card_tables[0].find('tr', class_='table_body_black') 
    table_description = card_tables[0].findAll('tr', class_='')
    card_with_hability = card_tables[1].find('td')

    
#Getting Title and Image of the Card...    
    title = table_header.find('b').getText()
    cardImage = table_image.find('a', class_='image')


#Getting Card Description...

    typeFlag = 0
    costFlag = 0
    frecFlag = 0
    editionFlag = 0
    strengthFlag = 0
    razaFlag = 0

    typeCard = ''
    strengthCard = ''
    costCard = ''
    frecCard = ''
    editionCard = ''
    razaCard = ''

    for row in table_description:
        th = row.find('th')
        if th.getText().strip() == "Tipo":
            #print('Type')
            typeCard = th.find_next('th').getText().strip('\n')
            typeFlag = 1
        elif th.getText().strip() == "Fuerza":
            #print('Strenght')
            strengthCard = th.find_next('th').getText().strip('\n')
            strengthFlag = 1
        elif th.getText().strip() == "Coste de oro":
            #print('Cost')
            costCard = th.find_next('th').getText().strip('\n')
            costFlag = 1
        elif th.getText().strip() == "Raza":
            #print('Raza')
            razaCard = th.find_next('th').getText().strip('\n')
            razaFlag = 1
        elif th.getText().strip() == "Frecuencia":
            #print('Frecuency')
            frecCard = th.find_next('th').getText().strip('\n')
            frecFlag = 1
        elif th.getText().strip() == "Edición":
            #print('Edition')
            editionCard = th.find_next('th').getText().strip('\n')
            editionFlag = 1

# Creating List with description parameters of the card and filling empty values...

    # Con Cost, Con Strenght, Con Raza
    if costFlag != 0 and strengthFlag != 0 and razaFlag != 0:
     cardDescription.append(typeCard)
     cardDescription.append(strengthCard)
     cardDescription.append(costCard)
     cardDescription.append(razaCard)
     cardDescription.append(frecCard)
     cardDescription.append(editionCard)
    # Sin costo, Con Fuerza, Con Raza    
    elif costFlag == 0 and strengthFlag != 0 and razaFlag != 0:
     cardDescription.append(typeCard)
     cardDescription.append(strengthCard)
     cardDescription.append('Sin Costo')
     cardDescription.append(razaCard)
     cardDescription.append(frecCard)
     cardDescription.append(editionCard)
    # Con costo, Sin Fuerza, Con Raza
    elif costFlag != 0 and strengthFlag == 0 and razaFlag != 0:
     cardDescription.append(typeCard)
     cardDescription.append('Sin Fuerza')
     cardDescription.append(costCard)
     cardDescription.append(razaCard)
     cardDescription.append(frecCard)
     cardDescription.append(editionCard)
    # Con costo, Con Fuerza, Sin Raza
    elif costFlag != 0 and strengthFlag != 0 and razaFlag == 0:
     cardDescription.append(typeCard)
     cardDescription.append(strengthCard)
     cardDescription.append(costCard)
     cardDescription.append('Sin Raza')
     cardDescription.append(frecCard)
     cardDescription.append(editionCard) 
    # Sin costo, Sin Fuerza, Raza
    elif costFlag == 0 and strengthFlag == 0 and razaFlag != 0:
     cardDescription.append(typeCard)
     cardDescription.append('Sin Fuerza')
     cardDescription.append('Sin Costo')
     cardDescription.append(razaCard)
     cardDescription.append(frecCard)
     cardDescription.append(editionCard)   
     # Sin costo, Con Fuerza, Sin Raza
    elif costFlag == 0 and strengthFlag != 0 and razaFlag == 0:
     cardDescription.append(typeCard)
     cardDescription.append(strengthCard)
     cardDescription.append('Sin Costo')
     cardDescription.append('Sin Raza')
     cardDescription.append(frecCard)
     cardDescription.append(editionCard)
    # Con costo, Sin Fuerza, Sin Raza
    elif costFlag != 0 and strengthFlag == 0 and razaFlag == 0:
     cardDescription.append(typeCard)
     cardDescription.append('Sin Fuerza')
     cardDescription.append(costCard)
     cardDescription.append('Sin Raza')
     cardDescription.append(frecCard)
     cardDescription.append(editionCard)  
    # Sin costo, Sin Fuerza, Sin Raza    
    elif costFlag == 0 and strengthFlag == 0 and razaFlag == 0:
     cardDescription.append(typeCard)
     cardDescription.append('Sin Fuerza')
     cardDescription.append('Sin Costo')
     cardDescription.append('Sin Raza')
     cardDescription.append(frecCard)
     cardDescription.append(editionCard)
    
    
    #Validating if card has hability...
    if (card_with_hability.getText().strip('\n') == "Habilidad"):
        #print('Hability')
        hability_flag = 1
        tr = card_tables[1].find('tr')
        td = tr.find('td')
        cardHability = td.find_next('td').text.strip()
        
        #Getting More Details of the card: Code, Artist and History...
        
        more_details = card_tables[2].find('tr', class_='table_body_black')
      
        for td in more_details.find('td'):
            cardCode = td.text.strip()
            cardArtist = td.find_next('td').text.strip()
            
            card_history = card_tables[3].find('tr', class_='table_body_black')
        for td in card_history.find('td'):
            cardHistory = td.text.strip()

        
    #Card Without Hability#            
    else: 
        #print('No Hability')
        cardHability = 'Sin Habilidad'
        more_details = card_tables[1].find('tr', class_='table_body_black')
    
        for td in more_details.find('td'):
            cardCode = td.text.strip()
            cardArtist = td.find_next('td').text.strip()

        card_history = card_tables[2].find('tr', class_='table_body_black')

        for td in card_history.find('td'):
            cardHistory = td.text.strip()

   #Putting all the card info in array...

    cardDetail.append((i, title, cardImage['href'], cardDescription[0], cardDescription[1], cardDescription[2], cardDescription[3], cardDescription[4], cardDescription[5], cardHability, cardCode, cardArtist, cardHistory))
    cardDescription = []
        
#It takes Time...
print(cardDetail)


[(0, 'Los Eddas', 'https://static.wikia.nocookie.net/myl-tcg/images/7/72/Los_Eddas.jpg/revision/latest?cb=20210107145032&path-prefix=es', 'Talisman', 'Sin Fuerza', '5', 'Sin Raza', 'Real', 'Ragnarok', 'Tu oponente se descarta de toda su mano, luego éste roba cuatro cartas.', '1 - 126 / Primera Edición', 'Mauricio Herrera', 'Antiguas voces al borde del olvido se escuchan en los versos y las prosas escritas en los Eddas. Se cantan: la creación del mundo, las historias de los dioses y las runas del Ragnarok.'), (1, 'Matador de Gigantes', 'https://static.wikia.nocookie.net/myl-tcg/images/3/34/Matador_de_Gigantes.jpg/revision/latest?cb=20210107150152&path-prefix=es', 'Talisman', 'Sin Fuerza', '5', 'Sin Raza', 'Real', 'Ragnarok', 'Todos tus aliados doblan su fuerza. Por turno.', '2 - 126 / Primera Edición', 'Mauricio Herrera', 'Thor, el protector de dioses y hombres, en su carro tirado por briosos machos cabríos, con feroz y poderoso furor arremete como el relámpago contra enemigos inmensos 

In [20]:
#Creating Dataframe...
df = pd.DataFrame(cardDetail)

#Adding Columns to dataFrame, defined in Global Variable...
df.columns = column_names
df.head(13)

,N°,Nombre,Imagen,Tipo,Fuerza,Coste de oro,Raza,Frecuencia,Edición,Habilidad,Código,Arte,Texto épico
0,0,Los Eddas,https://static.wikia.nocookie.net/myl-tcg/imag...,Talisman,Sin Fuerza,5,Sin Raza,Real,Ragnarok,"Tu oponente se descarta de toda su mano, luego...",1 - 126 / Primera Edición,Mauricio Herrera,Antiguas voces al borde del olvido se escuchan...
1,1,Matador de Gigantes,https://static.wikia.nocookie.net/myl-tcg/imag...,Talisman,Sin Fuerza,5,Sin Raza,Real,Ragnarok,Todos tus aliados doblan su fuerza. Por turno.,2 - 126 / Primera Edición,Mauricio Herrera,"Thor, el protector de dioses y hombres, en su ..."
2,2,Galdar,https://static.wikia.nocookie.net/myl-tcg/imag...,Talisman,Sin Fuerza,4,Sin Raza,Real,Ragnarok,Copia la habilidad de un Talismán que esté en ...,3 - 126 / Primera Edición,Gabriel Rodríguez,Sé un cántico de poder y varios más que presto...
3,3,Juramento de los Héroes,https://static.wikia.nocookie.net/myl-tcg/imag...,Talisman,Sin Fuerza,3,Sin Raza,Real,Ragnarok,Por cada aliado oponente que te haya atacado e...,4 - 126 / Primera Edición,Gabriel Rodríguez,"Porque es justo y necesario. Hay luchas duras,..."
4,4,Naglfar,https://static.wikia.nocookie.net/myl-tcg/imag...,Talismán,Sin Fuerza,4,Sin Raza,Real,Ragnarok,Todos los jugadores destruyen todas las cartas...,5 - 126 / Primera Edición,Gabriel Rodríguez,"El barco remachado, hecho de uñas de muerto, z..."
5,5,Espina Somnoliente,https://static.wikia.nocookie.net/myl-tcg/imag...,Talisman,Sin Fuerza,1,Sin Raza,Real,Ragnarok,Destruye un Arma o Tótem en juego y bota una c...,6 - 126 / Primera Edición,Pablo Santander,Con una espina la sumió en el sopor. Muchacha ...
6,6,Nidavellir,https://static.wikia.nocookie.net/myl-tcg/imag...,Totem,Sin Fuerza,4,Sin Raza,Real,Ragnarok,"Cuando tu oponente juegue un aliado, paga su v...",7 - 126 / Primera Edición,Mauricio Herrera,A nueve mundos llegué...Bajo la tierra y las p...
7,7,Lanza de Odín,https://static.wikia.nocookie.net/myl-tcg/imag...,Arma,Sin Fuerza,3,Sin Raza,Real,Ragnarok,Sólo puede ser llevada por una abominación; si...,8 - 126 / Primera Edición,Mauricio Herrera,"Lanza Gugnir es cosa admirable, forjada por Br..."
8,8,Martillo de Thor,https://static.wikia.nocookie.net/myl-tcg/imag...,Arma,Sin Fuerza,3,Sin Raza,Real,Ragnarok,Por cada punto de daño que haga este aliado al...,9 - 126 / Primera Edición,Mauricio Herrera,"Myöllnir, primer tesoro de Thor, cosa admirabl..."
9,9,Guantes de Hierro,https://static.wikia.nocookie.net/myl-tcg/imag...,Arma,Sin Fuerza,3,Sin Raza,Real,Ragnarok,Agrega +1 a la fuerza de su portador. Su porta...,10 - 126 / Primera Edición,Gabriel Rodríguez,"Tercer tesoro de Thor, cosa admirable. Al ceñi..."


In [24]:
#Exporting File to CSV...
#df.to_csv(fileName + '.csv', index=False)

#Exporting File to Excel
df.to_excel(fileName + '.xlsx', index=False)